## Feature Engineering

El código se enfoca en evaluar el sentimiento de cada revisión en base a si se recomienda o no, asignándole una puntuación de análisis de sentimientos. Antes de realizar este análisis, se aplica un proceso de stemming utilizando PorterStemmer, mediante la función apply_stemming, que actúa sobre la columna 'review' del DataFrame. Esto implica reducir las palabras a su forma base para capturar de manera más efectiva el significado general y disminuir la complejidad del texto. Se optó por el stemming debido a su rapidez, aunque es importante tener en cuenta que es un proceso más agresivo que la lematización. 

*Importamos las librerías necesarias para realizar el análisis.*

In [1]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer  # Importar el stemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer  # Importar el lematizador
import nltk


In [2]:
nltk.download('vader_lexicon')  # Descargar el modelo vader_lexicon de NLTK
nltk.download('punkt')  # Descargar el tokenizer de NLTK
nltk.download('wordnet')  # Descargar WordNet para lematización

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Cargar el dataset:\
*Cargamos el dataset en un DataFrame de pandas para poder manipularlo y realizar el análisis de sentimiento.*

In [3]:
df = pd.read_parquet(r'C:\Users\Usuario\Desktop\proyecto steam\Data Exportada\user_reviews_clean.parquet')

*Hacemos una copia para realizar las operaciones, mientras resguardamos los datos en el archivo original.*

In [4]:
df_copy = df.copy()
df_copy

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,True,:D


In [5]:
# Eliminar filas con valores nulos en la columna 'review'
#df = df.dropna(subset=['review'])

# Reemplazar expresiones
df_copy['review'] = df_copy['review'].replace(['10/10', '100/100'], 'Excelent', regex=True)

# Transformar todos los datos a tipo string
df_copy['review'] = df_copy['review'].astype(str)

Creamos una instancia para cada uno de los procesos donde: 
* Inicializamos el Analizador de Intensidad de Sentimiento (Sentiment Intensity Analyzer):
* Inicializamos el stemmer
* Inicializamos el lematizador


In [6]:
sid = SentimentIntensityAnalyzer()

stemmer = PorterStemmer()

lemmatizer = WordNetLemmatizer()  

Aplicación de Stemming a la Columna 'review' del DataFrame:\
*En este código, utilizamos el algoritmo de Stemming de NLTK para reducir las palabras a su forma base en la columna 'review' de nuestro DataFrame.*

In [7]:
# Definir la Función de Aplicación del Stemming
def apply_stemming(text):
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Aplicar la función de stemming a la columna 'review' del DataFrame
df_copy['review'] = df_copy['review'].apply(apply_stemming)

Aplicación de Lematización a la Columna 'review' del DataFrame:\
En este código, utilizamos el algoritmo de Stemming de NLTK para reducir las palabras a su forma base en la columna 'review' de nuestro DataFrame.

In [8]:
# Definir la Función de Aplicación de la Lematización
def apply_lemmatization(text):
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Aplicar la función de lematización a la columna 'review' del DataFrame
df_copy['review'] = df_copy['review'].apply(apply_lemmatization)

Aplicación del análisis de sentimiento:\
Creamos una función que tomará una reseña y aplicará el análisis de sentimiento, devolviendo el valor correspondiente según la escala especificada (malo='0', neutral='1' y positivo='2'). Si la reseña está ausente, la función devolverá '1'. Y luego la aplicamos al DataFrame

In [9]:
# Definir la Función de Análisis de Sentimiento
def get_sentiment_score(text, recommend):
    if pd.isnull(text):
        return 1
    sentiment_score = sid.polarity_scores(text)['compound']
    if recommend:
        sentiment_score += 0.5
    else:
        sentiment_score -= 0.5
    if sentiment_score <= -0.05:
        return 0
    elif sentiment_score >= 0.05:
        return 2
    else:
        return 1

# Aplicar la función get_sentiment_score a la columna de reseñas (review) para crear la nueva columna 'sentiment_analysis'.
df_copy['sentiment_analysis'] = df_copy.apply(lambda row: get_sentiment_score(row['review'], row['recommend']), axis=1)
df_copy.drop(columns=['review'], inplace=True)

* Analizamos cómo se correlacionan los sentimientos expresados en las reseñas con las recomendaciones dadas por los usuarios.
* Imprimimos en pantalla el conteo de las reseñas que muestran una inconsistencia entre el sentimiento expresado y la calificación dada por los usuarios, mostrando la cantidad de reseñas con sentimiento negativo y calificación positiva, así como la cantidad de reseñas con sentimiento positivo y calificación negativa.

In [10]:
# Reseñas con sentimiento positivo pero con una recomendación negativa.
positive_negative = df_copy[(df_copy['sentiment_analysis'] == 2) & (df_copy['recommend'] == False)]

# Reseñas con sentimiento negativo pero con una recomendación positiva.
negative_positive = df_copy[(df_copy['sentiment_analysis'] == 0) & (df_copy['recommend'] == True)]

# Conteo de reseñas con sentimiento negativo y recomendación positiva.
count_negative_positive = len(negative_positive)

# Conteo de reseñas con sentimiento positivo y recomendación negativa.
count_positive_negative = len(positive_negative)

print("Conteo de reseñas con sentimiento negativo y recomendación positiva:", count_negative_positive)
print("Conteo de reseñas con sentimiento positivo y recomendación negativa:", count_positive_negative)

Conteo de reseñas con sentimiento negativo y recomendación positiva: 2904
Conteo de reseñas con sentimiento positivo y recomendación negativa: 1323


Calculamos el porcentaje de errores respecto de la cantidad total de registros para conocer el margen de error del análisis de sentimiento.

In [11]:
total_mistakes = 2904 + 1323
total_records = 59333
percent_mistakes = (total_mistakes / total_records) * 100

print("Porcentaje de errores:", percent_mistakes)

Porcentaje de errores: 7.124197326951275


 Guardamos el DataFrame actualizado en un nuevo archivo .parquet

In [12]:
df_copy.to_parquet(r'C:\Users\Usuario\Desktop\proyecto steam\Sentiment_analysis\Sentiment_Analysis_parquet', index=False)